# Preprocessing data to graph structure

In [130]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [131]:
%reload_kedro

2022-08-31 09:23:44,675 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-31 09:23:44,835 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-31 09:23:45,058 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-31 09:23:45,060 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-31 09:23:45,094 - root - INFO - Registered line magic `run_viz`
2022-08-31 09:23:45,096 - root - INFO - Registered line magic `reload_kedro_mlflow`


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [132]:
from typing import Iterator, Tuple
import re

import pandas as pd
import numpy as np
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io.core import get_filepath_str

import gid_ml_framework.pipelines.santander_preprocessing.nodes
from gid_ml_framework.extras.datasets.chunks_dataset import (
 _load,
 _concat_chunks,
)

pd.options.mode.chained_assignment = None
pd.set_option('expand_frame_repr', True)
pd.set_option("display.max_rows", 999)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [158]:
customers_path = "santander_customers"
articles_path = "santander_articles"
transactions_train_path = "santander_transactions_train"
transactions_val_path = "santander_transactions_val"

In [159]:
customers = context.catalog.load(customers_path)
articles = context.catalog.load(articles_path)
transactions_train = _concat_chunks(context.catalog.load(transactions_train_path))
transactions_val = _concat_chunks(context.catalog.load(transactions_val_path))

2022-08-31 10:38:53,305 - kedro.io.data_catalog - INFO - Loading data from `santander_customers` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


2022-08-31 10:38:55,303 - kedro.io.data_catalog - INFO - Loading data from `santander_articles` (CSVDataSet)...
2022-08-31 10:38:55,345 - kedro.io.data_catalog - INFO - Loading data from `santander_transactions_train` (CSVDataSet)...
2022-08-31 10:38:55,467 - kedro.io.data_catalog - INFO - Loading data from `santander_transactions_val` (CSVDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/ked

In [135]:
customers.shape

(93146, 45)

In [111]:
articles.shape

(24, 1)

In [113]:
transactions_train.shape

(52432, 3)

In [115]:
transactions_val.shape

(3573, 3)

In [160]:
def _create_mapping(df: pd.DataFrame, map_column: str):
    """Creates mapping into consecutive integers for given column."""
    ids = df.loc[:, map_column].sort_values().reset_index(drop=True)
    mapping = {v: k for k, v in enumerate(ids)}
    return mapping

In [161]:
users_mapping = _create_mapping(customers, map_column="customer_id")
items_mapping = _create_mapping(articles, map_column="article_id")


In [151]:
set(transactions_train.customer_id).issubset(set(users_mapping.keys()))

True

In [162]:
transactions_train.replace({"customer_id": users_mapping, "article_id": items_mapping}, inplace=True)

In [153]:
set(transactions_train.customer_id).issubset(set(users_mapping.values()))

True

In [141]:
customers

,customer_id,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15911,F,ES,V,53,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,191298.656250,01 - TOP,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1
1,15913,F,ES,V,48,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,163073.187500,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1
2,15926,F,ES,V,56,1995-01-16,0,254,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,128376.242188,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,15950,N,ES,V,61,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,141979.265625,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1
4,15957,F,ES,V,54,1995-01-16,0,256,1,UNKNOWN,1,A,S,N,N,KAT,N,MADRID,1,155722.015625,01 - TOP,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93141,1553596,N,ES,H,47,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MALAGA,0,95195.492188,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93142,1553604,N,ES,H,66,2016-05-31,1,0,1,UNKNOWN,P,A,S,S,UNKNOWN,UNKNOWN,N,MALAGA,0,95195.492188,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93143,1553609,N,ES,V,43,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,ASTURIAS,0,86066.429688,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
93144,1553627,N,ES,V,31,2016-05-31,1,0,1,UNKNOWN,P,A,S,N,UNKNOWN,UNKNOWN,N,MADRID,0,138027.750000,UNKNOWN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [163]:
transactions_train.sort_values("customer_id")

,customer_id,date,article_id
11689,0,2015-04-28,11
11031,1,2015-11-28,7
14521,1,2016-01-28,12
23396,1,2016-01-28,22
4543,1,2015-12-28,2
...,...,...,...
10738,92341,2016-04-28,6
51518,92342,2016-04-28,20
6245,92342,2016-04-28,2
10739,92343,2016-04-28,6


In [140]:
customers.loc[customers.customer_id==92343, :]

,customer_id,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1


In [90]:
customers.shape[0]

92868

In [88]:
len(transactions.customer_id.unique())

19330

# Preprocessed data check

In [167]:
transactions_graph_path = "santander.santander_transactions_graph"
transactions_mapped_path = "santander_transactions_mapped"
users_mapping_path = "santander_users_mapping"
items_mapping_path = "santander_items_mapping"

In [168]:
transactions_graph = _concat_chunks(context.catalog.load(transactions_graph_path))
transactions_mapped = _concat_chunks(context.catalog.load(transactions_mapped_path))
users_mapping = context.catalog.load(users_mapping_path)
items_mapping = context.catalog.load(items_mapping_path)

2022-08-31 10:47:09,780 - kedro.io.data_catalog - INFO - Loading data from `santander.santander_transactions_graph` (CSVDataSet)...
2022-08-31 10:47:09,855 - kedro.io.data_catalog - INFO - Loading data from `santander_transactions_mapped` (CSVDataSet)...
2022-08-31 10:47:09,921 - kedro.io.data_catalog - INFO - Loading data from `santander_users_mapping` (PickleDataSet)...


/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/kedro/framework/context/context.py:488: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/home/michal/projects/gid-ml-framework/conf/base', '/home/michal/projects/gid-ml-framework/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")
/home/michal/projects/gid-ml-framework/.venv/lib/python3.8/site-packages/ked

2022-08-31 10:47:09,987 - kedro.io.data_catalog - INFO - Loading data from `santander_items_mapping` (PickleDataSet)...


In [169]:
transactions_graph

,user_id,item_id,time
0,366053,ind_cco_fin_ult1,1425081600
1,996556,ind_cco_fin_ult1,1425081600
2,1362626,ind_cco_fin_ult1,1425081600
3,456060,ind_cco_fin_ult1,1425081600
4,1218874,ind_cco_fin_ult1,1425081600
...,...,...,...
56000,1071812,ind_recibo_ult1,1464393600
56001,285488,ind_recibo_ult1,1464393600
56002,1401868,ind_recibo_ult1,1464393600
56003,1067005,ind_recibo_ult1,1464393600


In [170]:
transactions_mapped

,user_id,item_id,time
0,17074,2,1425081600
1,49210,2,1425081600
2,77896,2,1425081600
3,21642,2,1425081600
4,66652,2,1425081600
...,...,...,...
56000,54817,20,1464393600
56001,13213,20,1464393600
56002,80773,20,1464393600
56003,54368,20,1464393600


In [171]:
items_mapping

{'ind_ahor_fin_ult1': 0,
 'ind_aval_fin_ult1': 1,
 'ind_cco_fin_ult1': 2,
 'ind_cder_fin_ult1': 3,
 'ind_cno_fin_ult1': 4,
 'ind_ctju_fin_ult1': 5,
 'ind_ctma_fin_ult1': 6,
 'ind_ctop_fin_ult1': 7,
 'ind_ctpp_fin_ult1': 8,
 'ind_deco_fin_ult1': 9,
 'ind_dela_fin_ult1': 10,
 'ind_deme_fin_ult1': 11,
 'ind_ecue_fin_ult1': 12,
 'ind_fond_fin_ult1': 13,
 'ind_hip_fin_ult1': 14,
 'ind_nom_pens_ult1': 15,
 'ind_nomina_ult1': 16,
 'ind_plan_fin_ult1': 17,
 'ind_pres_fin_ult1': 18,
 'ind_reca_fin_ult1': 19,
 'ind_recibo_ult1': 20,
 'ind_tjcr_fin_ult1': 21,
 'ind_valo_fin_ult1': 22,
 'ind_viv_fin_ult1': 23}

In [165]:
users_mapping

{15911: 0,
 15913: 1,
 15926: 2,
 15950: 3,
 15957: 4,
 15976: 5,
 16000: 6,
 16002: 7,
 16003: 8,
 16007: 9,
 16024: 10,
 16042: 11,
 16055: 12,
 16060: 13,
 16067: 14,
 16093: 15,
 16102: 16,
 16113: 17,
 16125: 18,
 16162: 19,
 16201: 20,
 16217: 21,
 16255: 22,
 16261: 23,
 16311: 24,
 16329: 25,
 16412: 26,
 16456: 27,
 16463: 28,
 16529: 29,
 16572: 30,
 16580: 31,
 16584: 32,
 16590: 33,
 16595: 34,
 16641: 35,
 16650: 36,
 16666: 37,
 16680: 38,
 16681: 39,
 16720: 40,
 16723: 41,
 16730: 42,
 16747: 43,
 16749: 44,
 16757: 45,
 16758: 46,
 16770: 47,
 16820: 48,
 16826: 49,
 16834: 50,
 16841: 51,
 16848: 52,
 16849: 53,
 16861: 54,
 16883: 55,
 16898: 56,
 16940: 57,
 16959: 58,
 16973: 59,
 17010: 60,
 17045: 61,
 17098: 62,
 17183: 63,
 17201: 64,
 17215: 65,
 17279: 66,
 17337: 67,
 17360: 68,
 17370: 69,
 17417: 70,
 17422: 71,
 17443: 72,
 17479: 73,
 17493: 74,
 17523: 75,
 17535: 76,
 17536: 77,
 17571: 78,
 17591: 79,
 17601: 80,
 17622: 81,
 17645: 82,
 17653: 83,
 1